1) Реализовать простейший динамический массив, поддерживающий добавление элемента в конец массива при помощи метода append(e).
Также должны поддерживаться следующие операции:
извлечение элемента по индексу;
устанавка нового значения по индексу;
получение текущей длины массива.
1.1) Добавить реализацию удаления элемента из любого места массива и поддержку корректного вывода массива при помощи функции print.

In [33]:
class DynamicArray:
    def __init__(self):
        self.array = []

    def append(self, element):
        self.array.append(element)

    def __getitem__(self, index):
        return self.array[index]

    def __setitem__(self, index, value):
        self.array[index] = value

    def __len__(self):
        return len(self.array)

    def __str__(self):
        return str(self.array)

    def remove(self, index):
        del self.array[index]


# Тестирование
array = DynamicArray()
array.append(1)
array.append(2)
array.append(3)

print(array[1])  # 2
array[1] = 4
print(array)  # [1, 4, 3]
array.remove(1)
print(array)  # [1, 3]


2
[1, 4, 3]
[1, 3]


2) Реализовать генератор, который возвращает значение поочередно извлекаемое из конца двух очередей (в качестве очереди используется deque из collections). Если очередь из которой извлекается элемент пуста - генератор заканчивает работу.


In [34]:
from collections import deque

def merge_queues(queue1, queue2):
    while queue1 or queue2:
        if queue1:
            yield queue1.pop()
        if queue2:
            yield queue2.pop()


# Тестирование
queue1 = deque([1, 3, 5])
queue2 = deque([2, 4, 6])

for element in merge_queues(queue1, queue2):
    print(element)  # 6 5 4 3 2 1


5
6
3
4
1
2


3) Реализовать классы с медодом action():
а) Класс Pump - в методе action() извлекает очередное значение из генератора и помещает значение в очередь (очередь передается в конструктор).


In [35]:
from queue import Queue

class Pump:
    def __init__(self, queue, generator):
        self.queue = queue
        self.generator = generator

    def action(self):
        try:
            value = next(self.generator)
            self.queue.put(value)
            print(f"Value {value} put into the queue")
        except StopIteration:
            print("Generator is exhausted")

queue1 = Queue()
generator1 = (x for x in range(3))
pump1 = Pump(queue1, generator1)

pump1.action()
pump1.action()
pump1.action()
pump1.action()  


Value 0 put into the queue
Value 1 put into the queue
Value 2 put into the queue
Generator is exhausted


3) Реализовать классы с медодом action():
b) Класс MultiAction - при вызове метода action() n раз вызвает метод action() класса, переданного в конструкторе. Число n также определяется в конструкторе.

In [36]:
class MultiAction:
    def __init__(self, obj, n):
        self.obj = obj
        self.n = n

    def action(self):
        for _ in range(self.n):
            self.obj.action()

# Пример использования класса MultiAction
class MyClass:
    def action(self):
        print("Action performed")

obj = MyClass()
multi_action = MultiAction(obj, 3)

multi_action.action()


Action performed
Action performed
Action performed


3) Реализовать классы с медодом action():
c) Класс MultiPump - в методе action() извлекает очередное значение из генератора и помещает значение в одну из очередей (очереди передается в конструкторе); очереди , в которые помещаются очередные значения, меняются по порядку.

In [37]:
from collections import deque

class MultiPump:
    def __init__(self, generator, queues):
        self.generator = generator
        self.queues = queues
        self.current_queue_index = 0

    def action(self):
        try:
            value = next(self.generator)
            self.queues[self.current_queue_index].append(value)
            self.current_queue_index = (self.current_queue_index + 1) % len(self.queues)
        except StopIteration:
            pass


# Тестирование
generator = (i for i in range(5))
queue1 = deque()
queue2 = deque()

multi_pump = MultiPump(generator, [queue1, queue2])

multi_pump.action()
multi_pump.action()

print(queue1)  # [0, 2]
print(queue2)  # [1, 3]


deque([0])
deque([1])


4) При помощи GenFromQ, Pump реализовать систему опработки сообщений. Сообщения создаются генератором сообщений возвращающим случайным образорм одно из сообщений. Сообщения из трех генераторов закачиваются в три очереди при помощи классов Pump, далее при помощи GenFromQ и Pump объединяются в одну очередь и выводятся на экран (можно реализовать при помощи класса с action и вызываемого при помощи MultiPump).


In [38]:
import random
import time
from queue import Queue

class MessageGenerator:
    def generate_message(self):
        messages = ["Message 1", "Message 2", "Message 3"]
        return random.choice(messages)

class GenFromQ:
    def __init__(self, generator, queue):
        self.generator = generator
        self.queue = queue

    def action(self):
        message = self.generator.generate_message()
        self.queue.put(message)

class Pump:
    def __init__(self, queue):
        self.queue = queue

    def action(self):
        while not self.queue.empty():
            message = self.queue.get()
            print(message)

class MultiPump:
    def __init__(self, pumps):
        self.pumps = pumps

    def action(self):
        for pump in self.pumps:
            pump.action()

# Создание генераторов сообщений и очередей
generator1 = MessageGenerator()
generator2 = MessageGenerator()
generator3 = MessageGenerator()

queue1 = Queue()
queue2 = Queue()
queue3 = Queue()

# Создание объектов GenFromQ и Pump для каждого генератора и очереди
gen_from_q1 = GenFromQ(generator1, queue1)
gen_from_q2 = GenFromQ(generator2, queue2)
gen_from_q3 = GenFromQ(generator3, queue3)

pump1 = Pump(queue1)
pump2 = Pump(queue2)
pump3 = Pump(queue3)

# Объединение всех Pump в один MultiPump
multi_pump = MultiPump([pump1, pump2, pump3])

# Запуск генерации сообщений и их вывод на экран
for _ in range(5):  # Генерируем и выводим 5 сообщений
    gen_from_q1.action()
    gen_from_q2.action()
    gen_from_q3.action()
    multi_pump.action()




Message 1
Message 1
Message 2
Message 3
Message 1
Message 1
Message 1
Message 1
Message 3
Message 2
Message 2
Message 3
Message 2
Message 2
Message 1
